In [1]:
import pandas as pd
import gurobipy as gp

# Final Project Notebook

#### Marc Bitar, Chris Lee, Austin Douthit, Suren Dharmasena, Jack Casey

In [2]:
supply = dict({'SFO': 467,
               'LAX': 437})

demand = dict({'EWR': 356,
                'LGA': 516})

In [3]:
arcs, fuelCost = gp.multidict({
    ('SFO', 'ORD'): 8097.83,
    ('SFO', 'DEN'): 4253.45,
    ('LAX', 'DFW'): 5419.08,
    ('DFW', 'LGA'): 6096.46,
    ('LAX', 'ATL'): 8559.68,
    ('DEN', 'ORD'): 3897.16,
    ('DFW', 'DTW'): 3769.60,
    ('DEN', 'DFW'): 2819.50,
    ('ATL', 'IAD'): 2348.85,
    ('ORD', 'PHL'): 2982.25,
    ('DTW', 'PHL'): 1992.57,
    ('DTW', 'EWR'): 2146.52,
    ('PHL', 'EWR'): 351.89,
    ('ORD', 'LGA'): 3215.38,
    ('IAD', 'LGA'): 1007.28
})

# maxFlights is bigM
airport, hangerCost, maxFlights = gp.multidict({
    'DEN':	[100_000_000, 448],
    'DFW':	[130_000_000, 460],
    'ORD':	[215_000_000, 513],
    'DTW':	[80_000_000, 429],
    'ATL':	[150_000_000, 492],
    'PHL':	[90_000_000, 426],
    'IAD':	[105_000_000, 413]
})

In [4]:
#Model and variable creation
model = gp.Model('United')
flow = model.addVars(arcs, vtype='I', name="flow")
hangar = model.addVars(airport, vtype='B', name='hangar')

Restricted license - for non-production use only - expires 2024-10-28


In [5]:
#Defining objective
model.setObjective(gp.quicksum((fuelCost[arc]*flow[arc] for arc in arcs)) 
                   + gp.quicksum((hangerCost[a]*hangar[a] for a in airport)))

In [6]:
spots = supply.keys()
spots_flow = model.addConstrs((gp.quicksum(flow.select(spot, '*')) 
                                 <= supply[spot] for spot in spots), name="supply constraint")
#supply constraints

In [7]:
planes = demand.keys()
planes_flow = model.addConstrs((gp.quicksum(flow.select('*', plane)) - 
                                  gp.quicksum(flow.select(plane, '*')) >= demand[plane]
                                  for plane in planes), name="demand constraint")
#demand constraints

In [8]:
model.addConstrs(gp.quicksum(flow.select('*',j)) - gp.quicksum(flow.select(j,'*')) >= 0 for j in airport)
#transhipment Constraint

{'DEN': <gurobi.Constr *Awaiting Model Update*>,
 'DFW': <gurobi.Constr *Awaiting Model Update*>,
 'ORD': <gurobi.Constr *Awaiting Model Update*>,
 'DTW': <gurobi.Constr *Awaiting Model Update*>,
 'ATL': <gurobi.Constr *Awaiting Model Update*>,
 'PHL': <gurobi.Constr *Awaiting Model Update*>,
 'IAD': <gurobi.Constr *Awaiting Model Update*>}

In [9]:
# linking constraints
model.addConstr((flow[('SFO', 'DEN')] - maxFlights['DEN']*hangar['DEN'] <= 0), name = 'linking')
model.addConstr((flow[('LAX', 'DFW')] + flow[('DEN', 'DFW')] - maxFlights['DFW']*hangar['DFW'] <= 0), name = 'linking')
model.addConstr((flow[('SFO', 'ORD')] + flow[('DEN', 'ORD')] - maxFlights['ORD']*hangar['ORD'] <= 0), name = 'linking')
model.addConstr((flow[('DFW', 'DTW')] - maxFlights['DTW']*hangar['DTW'] <= 0), name = 'linking')
model.addConstr((flow[('DTW', 'PHL')] + flow[('ORD', 'PHL')] - maxFlights['PHL']*hangar['PHL'] <= 0), name = 'linking')
model.addConstr((flow[('LAX', 'ATL')] - maxFlights['ATL']*hangar['ATL'] <= 0), name = 'linking')
model.addConstr((flow[('ATL', 'IAD')] - maxFlights['IAD']*hangar['IAD'] <= 0), name = 'linking')

<gurobi.Constr *Awaiting Model Update*>

In [10]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18 rows, 22 columns and 47 nonzeros
Model fingerprint: 0x16e14ff4
Variable types: 0 continuous, 22 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [4e+02, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+02, 5e+02]
Found heuristic solution: objective 8.003705e+08
Presolve removed 8 rows and 10 columns
Presolve time: 0.01s
Presolved: 10 rows, 12 columns, 26 nonzeros
Found heuristic solution: objective 8.003504e+08
Variable types: 0 continuous, 12 integer (6 binary)
Found heuristic solution: objective 8.003482e+08

Root relaxation: objective 4.136570e+08, 11 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbe

In [11]:
#objective solution
model.printAttr('X')


    Variable            X 
-------------------------
flow[SFO,ORD]          436 
flow[LAX,DFW]          436 
flow[DFW,LGA]           80 
flow[DFW,DTW]          356 
flow[DTW,EWR]          356 
flow[ORD,LGA]          436 
 hangar[DFW]            1 
 hangar[ORD]            1 
 hangar[DTW]            1 


In [14]:
#objective value
model.objVal

434889133.96000004

In [13]:
model.write('test2.lp')